In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('blogs.csv')
df

Data              Labels
0     Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...         alt.atheism
1     Newsgroups: alt.atheism\nPath: cantaloupe.srv....         alt.atheism
2     Path: cantaloupe.srv.cs.cmu.edu!das-news.harva...         alt.atheism
3     Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...         alt.atheism
4     Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:53...         alt.atheism
...                                                 ...                 ...
1995  Xref: cantaloupe.srv.cs.cmu.edu talk.abortion:...  talk.religion.misc
1996  Xref: cantaloupe.srv.cs.cmu.edu talk.religion....  talk.religion.misc
1997  Xref: cantaloupe.srv.cs.cmu.edu talk.origins:4...  talk.religion.misc
1998  Xref: cantaloupe.srv.cs.cmu.edu talk.religion....  talk.religion.misc
1999  Xref: cantaloupe.srv.cs.cmu.edu sci.skeptic:43...  talk.religion.misc

[2000 rows x 2 columns]

In [3]:
df.head()

Data       Labels
0  Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...  alt.atheism
1  Newsgroups: alt.atheism\nPath: cantaloupe.srv....  alt.atheism
2  Path: cantaloupe.srv.cs.cmu.edu!das-news.harva...  alt.atheism
3  Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...  alt.atheism
4  Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:53...  alt.atheism

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Data    2000 non-null   object
 1   Labels  2000 non-null   object
dtypes: object(2)
memory usage: 31.4+ KB


In [5]:
import re
import nltk
from nltk.corpus import stopwords

In [27]:
nltk.download('stopwords') #Downlading stopwords if not already downloaded

#Clean and preprocess text
def preprocess_text(text):
    text=text.lower()
    text=re.sub(r'[^\w\s]', '',text)
    tokens=text.split()
    stop_words=set(stopwords.words('english'))
    tokens=[word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

#Apply preprocessing to the column
df['cleaned_text']=df['Data'].apply(preprocess_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer

#Intialize the TF-IDF vectorizer
tfidf=TfidfVectorizer()

#Transform the cleeaned text into a TF-IDF matrix
X=tfidf.fit_transform(df['cleaned_text'])
y=df['Labels']

In [29]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [30]:
from sklearn.naive_bayes import MultinomialNB
nb_model=MultinomialNB()
#Train the model
nb_model.fit(X_train,y_train)

y_pred=nb_model.predict(X_test)

In [31]:
from sklearn.metrics import accuracy_score,classification_report

accuracy=accuracy_score(y_test,y_pred)
print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(classification_report(y_test, y_pred))

Accuracy: 0.825
Classification Report:
                          precision    recall  f1-score   support

             alt.atheism       0.53      0.94      0.68        18
           comp.graphics       0.65      0.83      0.73        18
 comp.os.ms-windows.misc       0.95      0.86      0.90        22
comp.sys.ibm.pc.hardware       0.95      0.76      0.84        25
   comp.sys.mac.hardware       0.87      0.95      0.91        21
          comp.windows.x       1.00      0.80      0.89        25
            misc.forsale       0.92      0.67      0.77        18
               rec.autos       0.89      0.89      0.89        18
         rec.motorcycles       0.88      0.88      0.88        16
      rec.sport.baseball       0.80      0.89      0.84        18
        rec.sport.hockey       0.83      1.00      0.91        15
               sci.crypt       0.82      0.95      0.88        19
         sci.electronics       0.68      0.81      0.74        16
                 sci.med       0.94 

In [32]:
#pip install textblob

In [33]:
from textblob import TextBlob

# Perform sentiment analysis
def get_sentiment(text):
    sentiment = TextBlob(text).sentiment.polarity  # Returns a float between -1 (negative) and 1 (positive)
    if sentiment > 0:
        return 'positive'
    elif sentiment < 0:
        return 'negative'
    else:
        return 'neutral'

# Apply sentiment analysis to the cleaned text
df['sentiment'] = df['cleaned_text'].apply(get_sentiment)

In [34]:
# Analyze sentiment distribution across categories
sentiment_distribution = df.groupby('Labels')['sentiment'].value_counts().unstack()
print(sentiment_distribution)

sentiment                 negative  neutral  positive
Labels                                               
alt.atheism                   35.0      NaN      65.0
comp.graphics                 27.0      NaN      73.0
comp.os.ms-windows.misc       24.0      NaN      76.0
comp.sys.ibm.pc.hardware      19.0      NaN      81.0
comp.sys.mac.hardware         26.0      NaN      74.0
comp.windows.x                20.0      2.0      78.0
misc.forsale                  21.0      NaN      79.0
rec.autos                     24.0      NaN      76.0
rec.motorcycles               28.0      NaN      72.0
rec.sport.baseball            35.0      NaN      65.0
rec.sport.hockey              40.0      NaN      60.0
sci.crypt                     19.0      NaN      81.0
sci.electronics               25.0      NaN      75.0
sci.med                       34.0      NaN      66.0
sci.space                     28.0      NaN      72.0
soc.religion.christian        25.0      NaN      75.0
talk.politics.guns          

**Model Evaluation**
The evaluation of the model should cover:

The accuracy, precision, recall, and F1-score of the Naive Bayes classifier.
How the model's performance could potentially be improved (e.g., using different feature extraction techniques or tuning the model).
Insights into the sentiment analysis results and how sentiment varies across different categories.

**Report Writing**
Data Preprocessing: Document the text cleaning and feature extraction steps.
Naive Bayes Model: Explain how the Naive Bayes classifier was used, and discuss the evaluation metrics.
Sentiment Analysis: Discuss how sentiment was calculated and how it relates to blog categories.
Challenges and Improvements: Reflect on any challenges faced, such as handling imbalanced data or improving the model’s accuracy, and suggest ways to improve the model.
